# Retraining of the network

Use the already trained network as starting point and retrain it with new data to be more performant on this type of data

## Choose the retraining parameters

In [1]:
talkative = True  ## print info messages

from dextrusion.DialogParameters import DialogParameters
diag = DialogParameters()
diag.dialog_retrain()

nepochs = int(diag.epochs)
naug = float(diag.aug)
modeldir = diag.modelf
path = diag.path
validation_ratio = float(diag.ratio)

# The new model will be saved in the same directory as previous model with same name + "retrained"
newmodelpath = modeldir+'_retrained/'

## Load the previous network

In [ ]:
from dextrusion.DeXtrusion import DeXtrusion
dexter = DeXtrusion(verbose=talkative)
dexter.load_model(modeldir)
dexter.retrain_parameters( model_path=newmodelpath, datapath=path, aug=naug, epoch=nepochs )

## Load training and validation data

In [ ]:
from dextrusion.MovieGeneratorFromROI import MovieGeneratorFromROI
train = MovieGeneratorFromROI(
                 data_path = path,
                 batch_size=dexter.batch_size,
                 frame_shape=dexter.nframes,
                 win_halfsize = dexter.half_size,
                 val_ratio=validation_ratio,
                 balance=True,
                 ncat=dexter.ncat,
                 cat_names = dexter.catnames,
                 naug=naug,
                 augment_withnoise=True,
                 verbose=talkative)

if talkative:
    print("Training size: "+str(train.__len__()*dexter.batch_size))
    print("Nb of batchs: "+str(train.__len__()))

valid = train.get_validation_generator()
if talkative:
    print("Validation size: "+str(valid.__len__()*dexter.batch_size))

### ReTrain the network, and save it

In [ ]:
dexter.train_model(train_generator=train, validation_generator=valid, epochs=nepochs)  ## launch the retraining and save the new network
train.clean_tempdir()  ## remove the created files used for the training and validation